# Univariate Stochastic Volatility Model

For this short introduction, we follow http://www.chadfulton.com/topics/stochastic_volatility_qmle.html

\begin{align}
y_t & = e^{h_t / 2} \varepsilon_t & \varepsilon_t \sim N(0, 1)\\
h_{t+1} & = \mu + \phi(h_t - \mu) + \sigma_\eta \eta_t \qquad & \eta_t \sim N(0, 1) \\
h_1 & \sim N(\mu, \sigma_\eta^2 / (1 - \phi^2))
\end{align} 

where $y_t$ are mean-zero returns. 

A transform that was proposesed in Harvey et al. (1994) (for estimating vol of foreign exchange rates) reads: 

\begin{align}
\log y_t^2 & = h_t + \log \varepsilon_t^2\\
h_{t+1} & = \mu + \phi(h_t - \mu) + \sigma_\eta \eta_t \\
h_1 & \sim N(\mu, \sigma_\eta^2 / (1 - \phi^2))
\end{align} 

with $E(\log \varepsilon_t^2) \approx -1.27$ and $Var(\log \varepsilon_t^2) = \pi^2 / 2$.

**Quasi-Maximum Likelihood** "The quasi-likelihood approach (see e.g. Ruiz [1994] or Harvey et al. [1994]) replaces $\log \varepsilon_t^2$ with $E(\log \varepsilon_t^2) + \xi_t$ where $\xi_t \sim N(0, \pi^2 / 2)$. With this replacement, the model above becomes a linear Gaussian state space model and the Kalman filter can be used to compute the log-likelihoods using the prediction error decomposition.

Since $\log \varepsilon_t^2 - E(\log \varepsilon_t^2) \not \sim N(0, \pi^2 / 2)$, this is a quasi-maximum likelihood estimation routine that will only be valid asymptotically. However, Ruiz (1994) suggests that it may perform well nonetheless." (see http://www.chadfulton.com/topics/stochastic_volatility_qmle.html)

In [3]:
# set-up the SVM using http://www.chadfulton.com/topics/stochastic_volatility_qmle.html



import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.tools import (
    constrain_stationary_univariate, unconstrain_stationary_univariate)

# "Quasi-likelihood stochastic volatility"
class QLSV(sm.tsa.statespace.MLEModel):
    def __init__(self, endog):
        # Convert to log squares
        endog = np.log(endog**2)

        # Initialize the base model
        super(QLSV, self).__init__(endog, k_states=1, k_posdef=1,
                                   initialization='stationary')
        
        # Setup the observation covariance
        self['obs_intercept', 0, 0] = -1.27
        self['design', 0, 0] = 1
        self['obs_cov', 0, 0] = np.pi**2 / 2
        self['selection', 0, 0] = 1.
        
    @property
    def param_names(self):
        return ['phi', 'sigma2_eta', 'mu']
    
    @property
    def start_params(self):
        return np.r_[0.9, 1., 0.]
    
    def transform_params(self, params):
        return np.r_[constrain_stationary_univariate(params[:1]), params[1]**2, params[2:]]
    
    def untransform_params(self, params):
        return np.r_[unconstrain_stationary_univariate(params[:1]), params[1]**0.5, params[2:]]

    def update(self, params, **kwargs):
        super(QLSV, self).update(params, **kwargs)
        
        gamma = params[2] * (1 - params[0])
        self['state_intercept', 0, 0] = gamma
        self['transition', 0, 0] = params[0]
        self['state_cov', 0, 0] = params[1]